In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [2]:
cache_path = r"D:\TrainedModel"
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", cache_dir=cache_path)

In [3]:
model = PeftModel.from_pretrained(model, "D:\CoachBot\SFTuned")
tokenizer=AutoTokenizer.from_pretrained("D:\CoachBot\SFTuned", use_fast=False)

In [4]:
model = model.to("cuda")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=1024

In [ ]:
import torch

def chat():
    print("已加载模型，输入内容即可对话，输入“exit”或 Ctrl+C 退出。")
    while True:
        user_input = input("\n用户: ").strip()
        if not user_input or user_input.lower() == "exit":
            break

        # 构造 prompt
        prompt = f"用户：{user_input}\n系统："
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        # 
        generate_kwargs = {
            "max_new_tokens": 128,
            "temperature": 0.3,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": tokenizer.pad_token_id,
            "eos_token_id": tokenizer.eos_token_id,
        }
        with torch.no_grad():
            out = model.generate(**inputs, **generate_kwargs)

       
        generated = out[0][ inputs["input_ids"].shape[-1] : ]
        #reply = tokenizer.decode(generated, skip_special_tokens=True).strip()
        #reply = reply.split("用户：")[0].strip()
        text = tokenizer.decode(generated, skip_special_tokens=True)

        # 按“用户：”截断，只要系统的回答部分
        reply = text.split("用户：")[0].strip()
        print(f"系统: {reply}")

if __name__ == "__main__":
    chat()